In [1]:
import requests
import pandas as pd
from pathlib import Path

# Ensure data folder exists
Path('../data').mkdir(parents=True, exist_ok=True)

In [3]:
import httpx
import pandas as pd

# API setup
url = "https://api.cryptorank.io/v0/round/upcoming"
headers = {"Content-Type": "application/json"}
payload = {
    "limit": 10,
    "offset": 0
}

try:
    # Make the POST request
    response = httpx.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json().get("data", [])

    # Extract only the required fields
    simplified_data = []
    for item in data:
        simplified_data.append({
            "name": item.get("name"),
            "symbol": item.get("symbol"),
            "raise": item.get("raise"),
            "when": item.get("when"),
            "launchpad_name": item.get("launchpads", [{}])[0].get("name"),
            "launchpad_image": item.get("launchpads", [{}])[0].get("image")
        })

    # Convert to DataFrame
    df = pd.DataFrame(simplified_data)

    # Save to CSV
    csv_path = "../data/upcoming_ido.csv"
    df.to_csv(csv_path, index=False)


except httpx.HTTPError as e:
    print("Failed to fetch data:", e)


In [2]:
import requests
import pandas as pd

url = "https://api.cryptorank.io/v0/widgets/hot-events"
response = requests.get(url)
data = response.json()  # Already a list

hot_events = []
for item in data:  # No .get("data") here
    for backer in item.get("topFunds", []):
        hot_events.append({
            "project_key": item["key"],
            "project_name": item["name"],
            "project_logo": item["logo"],
            "totalRaise": item.get("totalRaise", 0),
            "taskTypes": ", ".join(item.get("taskTypes", [])),
            "backer_name": backer["name"],
            "backer_logo": backer["logo"]
        })

df_hot_rounds = pd.DataFrame(hot_events)
df_hot_rounds.to_csv("../data/hot_events.csv", index=False)
df_hot_rounds.head()

,project_key,project_name,project_logo,totalRaise,taskTypes,backer_name,backer_logo
0,huddle-01-activity764,Huddle01,https://img.cryptorank.io/coins/150x150.huddle...,2800000,"Social, Testnet, Bounty Platforms",Balaji Srinivasan,https://img.cryptorank.io/funds/60x60.balaji_s...
1,huddle-01-activity764,Huddle01,https://img.cryptorank.io/coins/150x150.huddle...,2800000,"Social, Testnet, Bounty Platforms",Stani Kulechov,https://img.cryptorank.io/funds/60x60.stani_ku...
2,huddle-01-activity764,Huddle01,https://img.cryptorank.io/coins/150x150.huddle...,2800000,"Social, Testnet, Bounty Platforms",LongHash Ventures,https://img.cryptorank.io/funds/60x60.longhash...
3,huddle-01-activity764,Huddle01,https://img.cryptorank.io/coins/150x150.huddle...,2800000,"Social, Testnet, Bounty Platforms",Protocol Labs,https://img.cryptorank.io/funds/60x60.protocol...
4,wunder-social-activity763,Wunder.Social,https://img.cryptorank.io/coins/150x150.wunder...,50000000,"Fill The Form, Bounty Platforms, Social",Rollman Management,https://img.cryptorank.io/funds/60x60.rollman_...


btc dominance

In [4]:
import os 
import httpx
import pandas as pd
output_dir = "../data/btc_dominance"
os.makedirs(output_dir, exist_ok=True)
periods = ["24H", "7D", "1M", "1Y"]

# Base URL
base_url = "https://api.cryptorank.io/v0/widgets/btc-dominance-chart?period="

# Save all period data to individual CSVs with millisecond timestamps
saved_csvs = []
for period in periods:
    url = f"{base_url}{period}"
    try:
        response = httpx.get(url, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        timestamps = json_data.get("timestamps", [])
        values = json_data.get("values", [])

        df = pd.DataFrame({
            "timestamp": pd.to_datetime(timestamps, unit="ms"),
            "dominance": values
        })

        csv_path = os.path.join(output_dir, f"btc_dominance_{period}.csv")
        df.to_csv(csv_path, index=False)
        saved_csvs.append(csv_path)
    except Exception as e:
        print(f"❌ Error fetching data for period {period}: {e}")

saved_csvs

['../data/btc_dominance\\btc_dominance_24H.csv',
 '../data/btc_dominance\\btc_dominance_7D.csv',
 '../data/btc_dominance\\btc_dominance_1M.csv',
 '../data/btc_dominance\\btc_dominance_1Y.csv']

In [7]:


url = "https://api.cryptorank.io/v0/widgets/fear-and-greed-index"
response = requests.get(url)
data = response.json()  # Already a list

fear_greed = {
    "today": data.get("today"),
    "yesterday": data.get("yesterday"),
    "lastWeek": data.get("lastWeek"),
    
    "lastMonth": data.get("lastMonth"),
    
}
df_fear_greed = pd.DataFrame([fear_greed])
df_fear_greed.to_csv("../data/data_fear_greed.csv", index=False)
df_fear_greed

,today,yesterday,lastWeek,lastMonth
0,43,25,30,45


In [11]:
from datetime import datetime, timedelta

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
group_by = 'month'  # or 'week'

# --- Cryptorank API URLs ---
FUNDRAISING_API = "https://api.cryptorank.io/v0/fund-chart/crypto-fundraising-activity?groupBy={group_by}&startDate={start_date}&endDate={end_date}"
IDO_API = "https://api.cryptorank.io/v0/public-raise-dashboard/raise?groupBy={group_by}&startDate={start_date}&endDate={end_date}"


# --- Fetch fundraising data ---
try:
    response = requests.get(FUNDRAISING_API.format(group_by=group_by, start_date=start_date, end_date=end_date))
    response.raise_for_status()
    fundraising_data = response.json()
    df_fundraising_trend = pd.DataFrame(fundraising_data)
    df_fundraising_trend.to_csv("../data/fundraising_trend.csv", index=False)
    print("Fundraising data saved successfully.")
except requests.RequestException as e:
    print(f"Error fetching fundraising data: {e}")
try:
    response = requests.get(IDO_API.format(group_by=group_by, start_date=start_date, end_date=end_date))
    response.raise_for_status()
    fundraising_data_ido = response.json()
    df_fundraising_ido = pd.DataFrame(fundraising_data_ido)
    df_fundraising_ido.to_csv("../data/fundraising_ido.csv", index=False)
    print("Fundraising data saved successfully.")
except requests.RequestException as e:
    print(f"Error fetching fundraising data: {e}")


Fundraising data saved successfully.
Fundraising data saved successfully.


In [9]:
import requests
import csv

# API endpoint and request
url = "https://api.cryptorank.io/v0/round/upcoming"
payload = {"limit": 100}
response = requests.post(url, json=payload)

# Accept 200 or 201
if response.status_code in [200, 201]:
    data = response.json().get("data", [])

    with open("../data/upcoming_ido.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["name" ,"image","symbol", "when", "raise", "launchpad", "launchpad_logo","category_name"])

        for item in data:
            name = item.get("name")
            image = item.get("image")
            symbol = item.get("symbol")
            when = item.get("when")
            raise_amount = item.get("raise")

            launchpads = item.get("launchpads", [])
            catergory_name = item.get("category")
           
            if launchpads:
                launchpad_name = launchpads[0].get("name", "N/A")
                launchpad_logo = launchpads[0].get("image", "N/A")
            else:
                launchpad_name = "N/A"
                launchpad_logo = "N/A"
            if catergory_name:
                catergory_name =catergory_name.get("name", "N/A")

            writer.writerow([name, image, symbol,when, raise_amount, launchpad_name, launchpad_logo,catergory_name])

    print("✅ Data saved to upcoming ido .csv")
else:
    print(f"❌ Error: Status code {response.status_code}")


✅ Data saved to upcoming ido .csv


In [14]:
import requests
import csv

# API endpoint and request
url = "https://api.cryptorank.io/v0/funding-rounds-v2"
payload = {"limit": 100}
response = requests.post(url, json=payload)

# Accept 200 or 201
if response.status_code in [200, 201]:
    data = response.json().get("data", [])

    with open("../data/upcoming_funds.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["name" ,"icon", "date", "raise", "fund_name", "fund_image","stage"])

        for item in data:
            name = item.get("name")
            icon = item.get("icon")
            date = item.get("date")
            raise_amount = item.get("raise")
            stage = item.get("stage")

            funds = item.get("funds", [])
           
            if funds:
                fund_name = funds[0].get("name", "N/A")
                fund_image = funds[0].get("image", "N/A")
            else:
                fund_name = "N/A"
                fund_image = "N/A"
           

            writer.writerow([name, icon,date, raise_amount, fund_name, fund_image,stage])

    print("✅ Data saved to upcoming-funds.csv")
else:
    print(f"❌ Error: Status code {response.status_code}")


✅ Data saved to upcoming-funds.csv
